In [7]:
import pandas as pd
from datetime import datetime

import yfinance as yf

import talib as ta

import warnings
warnings.filterwarnings("ignore")

In [8]:
# portfolio of stocks
stocks = ['^GDAXI']

In [ ]:

# Calculate dates properly
end_date = datetime.now()
start_date = end_date - pd.DateOffset(months=6)

# Convert to strings for yfinance if needed (yfinance accepts datetime objects too)
print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

# Download historical data from yf API
df = yf.download(stocks, start=start_date, end=end_date, group_by='ticker', interval='1h')

# # download data to csv
# df.to_csv("stocks_data.csv")

# # Load the CSV with MultiIndex columns (Tickers, OHLCV)
# df = pd.read_csv("stocks_data.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# # Show the result
# df.head()

Date range: 2025-03-11 to 2025-09-11


[*********************100%***********************]  1 of 1 completed



Ticker                           ^GDAXI                              \
Price                              Open          High           Low   
Datetime                                                              
2025-03-11 09:00:00+00:00  22772.230469  22835.099609  22729.880859   
2025-03-11 10:00:00+00:00  22771.179688  22777.140625  22688.339844   
2025-03-11 11:00:00+00:00  22716.589844  22716.589844  22638.500000   
2025-03-11 12:00:00+00:00  22698.210938  22701.470703  22603.199219   
2025-03-11 13:00:00+00:00  22664.910156  22708.699219  22510.279297   

Ticker                                          
Price                             Close Volume  
Datetime                                        
2025-03-11 09:00:00+00:00  22770.839844    0.0  
2025-03-11 10:00:00+00:00  22716.589844    0.0  
2025-03-11 11:00:00+00:00  22698.210938    0.0  
2025-03-11 12:00:00+00:00  22664.910156    0.0  
2025-03-11 13:00:00+00:00  22541.119141    0.0

In [11]:
# drop level 0 of the columns MultiIndex (the tickers)
df.columns = df.columns.droplevel(0)

# select only the 'Close' prices
df = df['Close']

df.head()

Datetime
2025-03-11 09:00:00+00:00    22770.839844
2025-03-11 10:00:00+00:00    22716.589844
2025-03-11 11:00:00+00:00    22698.210938
2025-03-11 12:00:00+00:00    22664.910156
2025-03-11 13:00:00+00:00    22541.119141
Name: Close, dtype: float64